In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

Firstly, ensure that all necessary libraries are installed. Code was enspried from:

https://www.youtube.com/watch?app=desktop&v=eovBbABk3hw


In [4]:
!pip install -q --upgrade pip
!pip install -q -U bitsandbytes
!pip install -q transformers
!pip install -q peft
!pip install -q accelerate
!pip install -q sentencepiece
!pip install -q langchain
!pip install -q farm-haystack[colab,faiss,inference]
!pip install -q unstructured
!pip install -q chromadb
!pip install -q faiss-cpu

Loading all the libraries

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
import pprint

from langchain import HuggingFacePipeline
from langchain.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.documents.base import Document

from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http

import re
import pandas as pd
import random

import numpy as np
np.random.seed(30224)

Loading the model config and model quantized from Huggingface

In [ ]:
#load bitsandbytesconfig with 4-bit quantisation
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
#load actual model with quant config
model = AutoModelForCausalLM.from_pretrained(
    'filipealmeida/Mistral-7B-Instruct-v0.1-sharded',
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00003-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
#load tokenizer for Mistral 7-b for Huggingface pipeline
tokenizer = AutoTokenizer.from_pretrained('filipealmeida/Mistral-7B-Instruct-v0.1-sharded')
tokenizer.bos_token_id = 1  # Set beginning of sentence token id

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

# Building the Huggingface Pipeline

In [ ]:
  llm_pipeline = transformers.pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    #temperatur=0.1,
    max_new_tokens=512,
    trust_remote_code=True
)

# Building the Langchain chain.
---

Code was inspried by:

https://python.langchain.com/docs/integrations/llms/huggingface_pipelines

In [ ]:
llm_langchain = HuggingFacePipeline(pipeline=llm_pipeline)

print(type(llm_langchain))

<class 'langchain_community.llms.huggingface_pipeline.HuggingFacePipeline'>


## Get sentence transformer Embedding from:
---
- https://api.python.langchain.com/en/stable/embeddings/langchain_community.embeddings.huggingface.HuggingFaceEmbeddings.html#langchain_community.embeddings.huggingface.HuggingFaceEmbeddings

- https://python.langchain.com/docs/integrations/platforms/huggingface

- https://huggingface.co/sentence-transformers/all-mpnet-base-v2

In [8]:
#'sentence-transformers/all-mpnet-base-v2'
Vectorizer = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Loading the documents:
!
# Important: Documents must be uploaded to the runtime environment first if run via Google Colab environment
---

In [9]:
def split_dora_by_article(text):
    # Regular expression pattern to match the paragraph headings (X.Y.)
    pattern = r'(?=Article\s\d+\n)'
    # Split the string via the regular expression
    parts = re.split(pattern, text)
    for i in range(len(parts)):
        parts[i] = parts[i].split("_____________", 1)[0]
    return parts[1:]
loader = PyPDFLoader('dora.pdf')
dora_pages = loader.load_and_split()

# Main text starts at page 43
dora_pages= dora_pages[43:]
content = ""
for page in dora_pages:
    content += page.page_content
dora=split_dora_by_article(content)
def get_dora_article_header(text):
    return text.split('\n')[0]


df_dora = pd.DataFrame(dora, columns=['text'])
df_dora['title'] = [get_dora_article_header(x) for x in dora]

df_dora

,text,title
0,Article 1\nSubject matter\n1. In order to achi...,Article 1
1,Article 2\nScope\n1. Without prejudice to para...,Article 2
2,Article 3\nDef initions\nFor the purposes of t...,Article 3
3,Article 4\nPropor tionality principle\n1. Fina...,Article 4
4,Article 5\nGovernance and organisation\n1. Fin...,Article 5
...,...,...
59,Article 60\nAmendments to Regulation (EU) No 6...,Article 60
60,Article 61\nAmendments to Regulation (EU) No 9...,Article 61
61,Article 62\nAmendments to Regulation (EU) No 6...,Article 62
62,Article 63\nAmendment to Regulation (EU) 2016/...,Article 63


In [10]:
def split_bait_by_paragraph(text):
    # Regular expression pattern to match the paragraph headings (X.Y.)
    pattern = r'(?<!\.\d)(?=\b\d+\.\d+\.\s)'

    # Remove all occurrances of the word "Circular 10/2017 (BA) in the version of 16.08.2021 Page X of 33" (which is part of the footer)
    tmp = text
    for i in range(1, 34):
        tmp = tmp.replace("Circular 10/2017 (BA) in the version of 16.08.2021 Page " + str(i) + " of 33", "")

    # Split the string via the regular expression
    parts = re.split(pattern, tmp)

    # Post-process to remove the last line of the previous part if the current part starts with X.1.
    for i in range(1, len(parts)):
        if parts[i].strip().startswith('1.'):
            parts[i-1] = parts[i-1].rsplit('\n', 1)[0]

    return parts[1:]
loader = PyPDFLoader("BAIT_en.pdf")
bait_pages=loader.load_and_split()
content = ""
for page in bait_pages:
    content += page.page_content
bait=split_bait_by_paragraph(content)
def get_bait_article_header(text):
    return text.split(' ')[0]
df_bait = pd.DataFrame(bait, columns=['text'])
df_bait['title'] = [get_bait_article_header(x) for x in bait]
df_bait

,text,title
0,1.1. The IT strategy shall fulfil the requirem...,1.1.
1,1.2. The management board shall define an IT s...,1.2.
2,2.1. IT governance is the structure used to ma...,2.1.
3,2.2. The management board is responsible for e...,2.2.
4,2.3. The institution shall ensure that appropr...,2.3.
...,...,...
82,12.1. Against the backdrop of the other chapte...,12.1.
83,12.2. The extent of critical infrastructure w...,12.2.
84,12.3. The information risk management and info...,12.3.
85,12.4. The CIP objective shall always be taken...,12.4.


In [11]:
#output text to see if loading worked
df_bait['text'][84]

'12.3. The information risk management and information security manage-\nment under chapters 3 and 4 of the BAIT shall take the CIP objective\ninto account and adopt measures to ensure that it is achieved. In par-\nticular, risks that have the potential to impair critical services to a sig-\nnificant degree shall be reduced using appropriate measures for risk\nmitigation and risk avoidance to a level appropriate for the CIP objec-\ntive. Measures which are able to counter the risks to availability when\nthe need for protection is high or very high are particularly suitable\nfor this. Among other things, high availability concepts should there-\nfore be examined and, if appropriate, applied.In principle, measures shall be taken to mitigate risks. This should involve\nmaintaining state of the art technology.\nThe required effort and expenditure should be proportionate to the conse-\nquences of the critical infrastructure concerned failing or being impaired.\nThis means that while risks c

## Necessary Preprocessing because of the Embedding:
---
Splitting all texts wich exceed 384 words into smaller bits and adding "paragraph number continued: " to the start of the spit texts as to inform the model of the texts origins if they should be retrieved during similarity search

In [12]:
def split_long_texts(df, column_name, word_limit):
    new_rows = []
    for j, row in df.iterrows():
        text = row[column_name]
        words = text.split()

        if len(words) > word_limit:
            for i in range(0, len(words), word_limit):
                new_row = row.copy()
                new_row[column_name] = ' '.join(words[i:i+word_limit])
                if i > 0:
                    new_row[column_name] = str(df["title"][j]) + " contd.: "+ new_row[column_name]
                new_rows.append(new_row)
        else:
            new_rows.append(row)
    return pd.DataFrame(new_rows)

In [13]:
df_dora_split = split_long_texts(df_dora, 'text', 381)
#making a quick check if the splitting-method actually works by quecking the length of the new elements
c2=0
for i, text in enumerate(df_dora_split['text']):
    if len(text.split()) > 384:
        print(f'{df_dora_split["title"][i]} from {df_dora_split["origin"][i]} exeeds 384 words! Length: {len(text.split())}')
        c2+=1
print(c2)

0


In [14]:
df_bait_split = split_long_texts(df_bait, 'title', 381)

In [15]:
split_dora_pages = []

# Iterate through the DataFrame column and create langchain Document objects
# which can be passed onto the VectorStore
for i, text in enumerate(df_dora_split['text']):

    document = Document(page_content=text, metadata={"source": "dora"})
    split_dora_pages.append(document)

## Vectorstore
---
Code inspired by:

https://python.langchain.com/docs/integrations/vectorstores/faiss

In [16]:
dora_vectorstore = FAISS.from_documents(documents=split_dora_pages, embedding=Vectorizer)

In [17]:
dora_retriever = dora_vectorstore.as_retriever()

In [18]:
dora_vectorstore.as_retriever()

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f676ebb0ca0>)

# Building the Prompt Template

In [ ]:
template = '''
  You are a polite and eager assistant. Answer the question based only on the following context:

  {context}

  Question: {question}

'''

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

# Create the Chain with Langchain Expression Langauge (LCEL)

In [ ]:
output_parser = StrOutputParser()

In [ ]:
dora_setup_and_retrieval = RunnableParallel(
    {'context': dora_retriever, 'question': RunnablePassthrough()}
)

In [ ]:
dora_chain =  dora_setup_and_retrieval | prompt | llm_langchain | output_parser

# Executing the main routine of the analysis:

In [ ]:
article_iteration_prompt = """
Which paragraph or paragraphs from DORA is most closely related to the following BAIT article: <<X>> ?
"""

bd_mapping = {}
for i, text in enumerate(df_bait_split['text']):
  print(i)
  title = df_bait_split['title'][i]
  aip = article_iteration_prompt.replace('<<X>>', text)
  bd_mapping[title] = dora_chain.invoke(aip)
  with open(f'new_bd_mapping_{i}', 'w') as f:
    f.write(bd_mapping[title].encode('ascii', 'ignore').decode('ascii'))
    f.close()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

84


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


85


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


86


In [ ]:
import json
with open('new_bd_iterative_response.json', 'w') as f:
  json.dump(bd_mapping, f)

manually rerun the pipeline for the empty responses. The process was stopped after the first four documents did not contain any content yet again.

In [ ]:
article_iteration_prompt = """
Which paragraph or paragraphs from DORA is most closely related to the following BAIT article: <<X>> ?
"""

bd_mapping = {}
for title in ["1.1.", "1.2.", "3.3.", "4.10.", "5.4.", "5.6.", "6.1.", "6.5.", "6.8.", "7.5.", "7.14.", "8.3.", "8.4.", "8.8.", "9.5.", "10.5.", "11.4.", "11.8."]:
  i = df_bait_split.index[df_bait_split['title']==title][0]
  print(i)
  text = df_bait_split['text'][i]

  aip = article_iteration_prompt.replace('<<X>>', text)
  bd_mapping[title] = dora_chain.invoke(aip)
  with open(f'post_bd_mapping_{i}', 'w') as f:
    f.write(bd_mapping[title].encode('ascii', 'ignore').decode('ascii'))
    f.close()

In [ ]:
import json
with open('post_bd_iterative_response.json', 'w') as f:
  json.dump(bd_mapping, f)

took about 45sec per chain invocation, suggesting a time of about 45*87 = 1h 5min total

## utilizing the VectorStore directly in order to evaluate the interesting response to BAIT pargraph 2.4

In [21]:
df_bait_split['text'][5]

'2.4. Conflicts of interest and activities that are not compatible with each\nother shall be avoided within the organisational and operational\nstructure of IT.Conflicts of interest between activities connected, for example, with applica-\ntion development and tasks performed by IT operations can be countered\nby taking organisation or operational measures and/or by defining roles ad-\nequately.Information risk management\n'

In [22]:
dora_vectorstore.similarity_search_with_relevance_scores(
    f"{df_bait_split['text'][5]}"
)

[(Document(page_content='Article 6 ICT risk management framew ork 1. Financial entities shall have a sound, compreh ensive and well-documente d ICT risk management framew ork as part of their overall risk manage ment system, which enables them to address ICT risk quic kly, efficiently and compr ehensively and to ensure a high level of digital operational resilience. 2. The ICT risk management framework shall include at least strategies, policies, procedures, ICT protocols and tools that are necessar y to duly and adequat ely prot ect all information assets and ICT assets, including comput er software, hardware, servers, as well as to prote ct all relevant physical compo nents and infrastr uctures, such as premises, data centres and sensitive designated areas, to ensure that all information assets and ICT assets are adequately prot ected from risks including damage and unauthor ised access or usage. 3. In accordance with their ICT risk management framew ork, financial entities shall min